In [58]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

# Pré-Processamento dos Áudios

### Conversão para o formato .wav

Primeiro é preciso converter os arquivos .au para o formato .wav

In [42]:
for dir_ in os.listdir('genres'):
    for file_ in os.listdir('genres/'+dir_):
        os.system('sox ' + 'genres/'+dir_+'/'+file_+' ' + 'sound_input/'+dir_+'/'+file_[:file_.find('au')-1]+'.wav')

### Extração das Features

Após converter os audios para um formato que possa ser manipulado pelo Python iremos extrair informações importantes a respeito de cada som e montar nosso dataframe que será usado como input dos modelos de aprendizado de máquina.

In [55]:
# import das bibliotecas usadas nessa etapa de extracao das features
from scipy.stats import kurtosis
from scipy.stats import skew
import librosa

In [44]:
# dicionario para mapear cada som como um valor numérico (a fim de facilitar a classificacao)
genres = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4, 
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

As features que serão extraídas de cada uma das músicas são:
1. **Spectral Centroid**

> Indica onde o “centro de massa” de um som está localizado e é calculado como a média ponderada das freqüências presentes no som.

2. **Spectral Rollof**

> É uma medida da forma do sinal. Representa a frequência abaixo da qual uma percentagem especificada da energia espectral total, p. 85%, estará situada.

3. **Zero Crossing Rate**

> O *Zero Crossing Rate* é a taxa de mudanças de sinal ao longo de um sinal, ou seja, a taxa na qual o sinal muda de positivo para negativo ou de volta.

4. **Root Mean Square**

> É uma média dos valores de amplitude ao longo do tempo, o que possibilita uma maneira mais precisa de medir o volume em relação à escala digital completa.

5. **Spectral Flux**

> É uma medida da rapidez com que o espectro de potência de um sinal está mudando, calculado pela comparação do espectro de energia de um quadro com o espectro de potência do quadro anterior.

6. **Chroma features**

> Os *Chroma features* são uma representação interessante e poderosa para o áudio de música, no qual todo o espectro é projetado em 12 faixas, representando os 12 semitons (ou cromas) distintos da oitava musical.


7. **Spectral Bandwidth**

> É o intervalo de comprimento de onda no qual uma quantidade espectral irradiada não é menor que a metade do seu valor máximo. É uma medida da extensão do espectro.

8. **Mel Frequency Cepstral Coefficients (MFCCs)**

> Os *Mel frequency cepstral coefficients (MFCCs)* de um sinal são um pequeno conjunto de características (geralmente em torno de 10–20) que descrevem forma geral de um envelope espectral. Ele modela as características da voz humana.


In [45]:
def get_features(y, sr, n_fft = 1024, hop_length = 512):
    """
        :param: y - series temporais do audio
        :param: sr - taxa de amostragem das series temporais
        :param: n_fft - tamanho da janela da FFT(Transformada de Fourier)
        :param: hop_length - numero de amostras entre quadros sucessivos
        
        :return: features - dicionario com cada uma das features extraidas da musica
    """
    # features que serao usadas
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None,
                'chroma':None, 'bandwidth': None}
    
    # chama as funcoes do librosa que calcula cada uma das features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rmse(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()

    # tratamento especial para o MFCC
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # pega as estatisticas de cada uma das features
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [60]:
# loop que itera sobre todos os arquivos e calcula as suas respectivas features
arr_features = []
for dir_ in os.listdir('sound_input'):
    if '.DS_' in dir_:
        os.remove('sound_input/'+dir_)
    else:
        for file_ in os.listdir('sound_input/'+dir_):
            file_name = 'sound_input/'+dir_+'/'+file_
            if '.DS_' in file_name:
                os.remove(file_name)
            else:
                print("Reading file: {}".format(file_name))
                signal, sr = librosa.load(file_name)
                
                features = get_features(signal, sr)
                features['genre'] = genres[dir_]

                arr_features.append(features)

Reading file: sound_input/pop/pop.00027.wav
Reading file: sound_input/pop/pop.00033.wav
Reading file: sound_input/pop/pop.00032.wav
Reading file: sound_input/pop/pop.00026.wav
Reading file: sound_input/pop/pop.00030.wav
Reading file: sound_input/pop/pop.00024.wav
Reading file: sound_input/pop/pop.00018.wav
Reading file: sound_input/pop/pop.00019.wav
Reading file: sound_input/pop/pop.00025.wav
Reading file: sound_input/pop/pop.00031.wav
Reading file: sound_input/pop/pop.00009.wav
Reading file: sound_input/pop/pop.00035.wav
Reading file: sound_input/pop/pop.00021.wav
Reading file: sound_input/pop/pop.00020.wav
Reading file: sound_input/pop/pop.00034.wav
Reading file: sound_input/pop/pop.00008.wav
Reading file: sound_input/pop/pop.00022.wav
Reading file: sound_input/pop/pop.00036.wav
Reading file: sound_input/pop/pop.00037.wav
Reading file: sound_input/pop/pop.00023.wav
Reading file: sound_input/pop/pop.00044.wav
Reading file: sound_input/pop/pop.00050.wav
Reading file: sound_input/pop/po

Reading file: sound_input/metal/metal.00003.wav
Reading file: sound_input/metal/metal.00017.wav
Reading file: sound_input/metal/metal.00016.wav
Reading file: sound_input/metal/metal.00002.wav
Reading file: sound_input/metal/metal.00028.wav
Reading file: sound_input/metal/metal.00014.wav
Reading file: sound_input/metal/metal.00000.wav
Reading file: sound_input/metal/metal.00001.wav
Reading file: sound_input/metal/metal.00015.wav
Reading file: sound_input/metal/metal.00029.wav
Reading file: sound_input/metal/metal.00011.wav
Reading file: sound_input/metal/metal.00005.wav
Reading file: sound_input/metal/metal.00039.wav
Reading file: sound_input/metal/metal.00038.wav
Reading file: sound_input/metal/metal.00004.wav
Reading file: sound_input/metal/metal.00010.wav
Reading file: sound_input/metal/metal.00006.wav
Reading file: sound_input/metal/metal.00012.wav
Reading file: sound_input/metal/metal.00013.wav
Reading file: sound_input/metal/metal.00007.wav
Reading file: sound_input/disco/disco.00

Reading file: sound_input/blues/blues.00008.wav
Reading file: sound_input/blues/blues.00036.wav
Reading file: sound_input/blues/blues.00022.wav
Reading file: sound_input/blues/blues.00023.wav
Reading file: sound_input/blues/blues.00037.wav
Reading file: sound_input/blues/blues.00012.wav
Reading file: sound_input/blues/blues.00006.wav
Reading file: sound_input/blues/blues.00007.wav
Reading file: sound_input/blues/blues.00013.wav
Reading file: sound_input/blues/blues.00005.wav
Reading file: sound_input/blues/blues.00011.wav
Reading file: sound_input/blues/blues.00039.wav
Reading file: sound_input/blues/blues.00038.wav
Reading file: sound_input/blues/blues.00010.wav
Reading file: sound_input/blues/blues.00004.wav
Reading file: sound_input/blues/blues.00028.wav
Reading file: sound_input/blues/blues.00000.wav
Reading file: sound_input/blues/blues.00014.wav
Reading file: sound_input/blues/blues.00015.wav
Reading file: sound_input/blues/blues.00001.wav
Reading file: sound_input/blues/blues.00

Reading file: sound_input/classical/classical.00047.wav
Reading file: sound_input/classical/classical.00053.wav
Reading file: sound_input/classical/classical.00057.wav
Reading file: sound_input/classical/classical.00043.wav
Reading file: sound_input/classical/classical.00094.wav
Reading file: sound_input/classical/classical.00080.wav
Reading file: sound_input/classical/classical.00081.wav
Reading file: sound_input/classical/classical.00095.wav
Reading file: sound_input/classical/classical.00042.wav
Reading file: sound_input/classical/classical.00056.wav
Reading file: sound_input/classical/classical.00040.wav
Reading file: sound_input/classical/classical.00054.wav
Reading file: sound_input/classical/classical.00068.wav
Reading file: sound_input/classical/classical.00083.wav
Reading file: sound_input/classical/classical.00097.wav
Reading file: sound_input/classical/classical.00096.wav
Reading file: sound_input/classical/classical.00082.wav
Reading file: sound_input/classical/classical.00

Reading file: sound_input/rock/rock.00043.wav
Reading file: sound_input/rock/rock.00057.wav
Reading file: sound_input/rock/rock.00080.wav
Reading file: sound_input/rock/rock.00094.wav
Reading file: sound_input/rock/rock.00030.wav
Reading file: sound_input/rock/rock.00024.wav
Reading file: sound_input/rock/rock.00018.wav
Reading file: sound_input/rock/rock.00019.wav
Reading file: sound_input/rock/rock.00025.wav
Reading file: sound_input/rock/rock.00031.wav
Reading file: sound_input/rock/rock.00027.wav
Reading file: sound_input/rock/rock.00033.wav
Reading file: sound_input/rock/rock.00032.wav
Reading file: sound_input/rock/rock.00026.wav
Reading file: sound_input/rock/rock.00022.wav
Reading file: sound_input/rock/rock.00036.wav
Reading file: sound_input/rock/rock.00037.wav
Reading file: sound_input/rock/rock.00023.wav
Reading file: sound_input/rock/rock.00009.wav
Reading file: sound_input/rock/rock.00035.wav
Reading file: sound_input/rock/rock.00021.wav
Reading file: sound_input/rock/roc

Reading file: sound_input/country/country.00038.wav
Reading file: sound_input/country/country.00010.wav
Reading file: sound_input/country/country.00004.wav
Reading file: sound_input/country/country.00009.wav
Reading file: sound_input/country/country.00021.wav
Reading file: sound_input/country/country.00035.wav
Reading file: sound_input/country/country.00034.wav
Reading file: sound_input/country/country.00020.wav
Reading file: sound_input/country/country.00008.wav
Reading file: sound_input/country/country.00036.wav
Reading file: sound_input/country/country.00022.wav
Reading file: sound_input/country/country.00023.wav
Reading file: sound_input/country/country.00037.wav
Reading file: sound_input/country/country.00033.wav
Reading file: sound_input/country/country.00027.wav
Reading file: sound_input/country/country.00026.wav
Reading file: sound_input/country/country.00032.wav
Reading file: sound_input/country/country.00024.wav
Reading file: sound_input/country/country.00030.wav
Reading file

In [65]:
# converte a lista de dicionarios em um dataframe do pandas
df_features = pd.DataFrame(arr_features)
df_features.head()

,bandwidth_kurtosis,bandwidth_mean,bandwidth_skew,bandwidth_std,centroid_kurtosis,centroid_mean,centroid_skew,centroid_std,chroma_kurtosis,chroma_mean,...,rmse_std,roloff_kurtosis,roloff_mean,roloff_skew,roloff_std,tempo,zcr_kurtosis,zcr_mean,zcr_skew,zcr_std
0,-0.381434,2843.572866,-0.109249,487.839032,1.034074,2505.854977,0.800053,935.467392,-0.995680,0.455167,...,0.060098,-0.815526,5534.382930,-0.115232,2162.745538,83.354335,4.234907,0.101837,1.593724,0.061233
1,0.239228,2697.859530,-0.477103,429.705382,6.413877,2747.695023,1.356641,679.724841,-1.035169,0.475538,...,0.052960,0.086089,5792.881290,-0.243072,1507.452974,99.384014,19.048932,0.136982,3.037529,0.055344
2,0.555687,3371.636447,-0.159385,362.150416,0.293103,4037.961443,0.504675,1357.694097,-1.004613,0.471255,...,0.050640,2.039517,8214.009367,-1.375242,1735.986006,95.703125,1.323397,0.228610,1.142884,0.151633
3,-0.751230,2744.526028,0.404663,546.557912,3.341373,2344.006310,1.662924,1006.785294,-0.577526,0.377737,...,0.094043,-0.945674,5048.328741,0.458483,2325.375690,151.999081,16.673878,0.103481,3.315788,0.069092
4,0.348567,2931.136087,0.242101,371.982073,2.458288,2945.478205,1.185425,811.953320,-0.482446,0.393403,...,0.085871,-0.244253,6392.563511,-0.046291,1611.131650,95.703125,7.592469,0.148665,2.136062,0.075436


In [66]:
# salva o dataframe como um arquivo .csv
df_features.to_csv('features.csv', sep=';', index=False)